In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [0]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [0]:
from package import MakeFeatures, make_predict

R_STATE = 17

In [0]:
PATH_ANS = 'csv'

if not os.path.exists(PATH_ANS):
    os.mkdir(PATH_ANS)

In [0]:
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')

In [7]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [0]:
X_train, y_train = train.drop(['dep_delayed_15min'], axis=1), train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [9]:
mf = MakeFeatures(is_weekend=1, 
                  seasons=1, 
                  day_of_month=1, 
                  dep_capital=1, 
                  arr_capital=1, 
                  route=1, 
                  log_dist=1, 
                  uc=1, 
                  time = 1,
                  new_dep_time = 1,
                  hour_imp = 1)
mf.fit(X_train)

X_train_new = mf.fit_transform(X_train)
X_train_new.head()

Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Hour_sq-s feature complete!
Route complete!
Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Hour_sq-s feature complete!
Route complete!


,Seasons_cos,Seasons_sin,IsWeekend,Day,DepFromCap,ArrForCap,LogDist,Hour,Minute,Hour_sq,Hour_sq2,AA,AQ,AS,B6,CO,DH,DL,EV,F9,FL,HA,HP,MQ,NW,OH,OO,TZ,UA,US,WN,XE,YV,ABE_Origin,ABI_Origin,ABQ_Origin,ABY_Origin,ACK_Origin,ACT_Origin,ACV_Origin,...,SMF_Dest,SMX_Dest,SNA_Dest,SOP_Dest,SPI_Dest,SPS_Dest,SRQ_Dest,STL_Dest,STT_Dest,STX_Dest,SUN_Dest,SWF_Dest,SYR_Dest,TEX_Dest,TLH_Dest,TOL_Dest,TPA_Dest,TRI_Dest,TTN_Dest,TUL_Dest,TUP_Dest,TUS_Dest,TVC_Dest,TWF_Dest,TXK_Dest,TYR_Dest,TYS_Dest,VCT_Dest,VIS_Dest,VLD_Dest,VPS_Dest,WRG_Dest,WYS_Dest,XNA_Dest,YAK_Dest,YUM_Dest,day,evening,morning,night
0,-5.000000e-01,-0.866025,1,21,0,0,6.595781,19,34,361,130321,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,-5.000000e-01,0.866025,0,20,0,0,6.726233,15,48,225,50625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,-1.836970e-16,-1.000000,0,2,0,0,6.030685,14,22,196,38416,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,8.660254e-01,-0.500000,1,25,0,0,6.770789,10,15,100,10000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5.000000e-01,-0.866025,1,7,0,0,6.047372,18,28,324,104976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train_new, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=R_STATE)

In [0]:
X_train_xgb, X_train_logreg, y_train_xgb, y_trian_logreg = X_train_part, X_train_part, y_train_part, y_train_part

# XGBoost Classifier training

In [0]:
xgb = XGBClassifier(learning_rate=0.1,
                        max_depth=4,
                        min_child_weight=2,
                        n_estimators=1000,
                        gamma=0.1,
                        subsamle = 0.9,
                        colsample_bytree = 0.5,
                        reg_alpha = 1.4,
                        objective= 'binary:logistic'
                        )

In [13]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=R_STATE)
cross_val_score(xgb, X=X_train_xgb.values, y=y_train_xgb, cv=skf, scoring="roc_auc", verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 18.4min finished


array([0.73926331, 0.73293426, 0.74023485])

In [15]:
xgb.fit(X_train_xgb.values, y_train_xgb)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.1,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=2, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=1.4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsamle=0.9, subsample=1, verbosity=1)

In [16]:
predict_xgb = xgb.predict_proba(X_valid.values)[:, 1]
roc_auc_score(y_valid, predict_xgb)

0.737582004351949

# Logistic Regression training

In [0]:
sc = StandardScaler()
X_train_logreg_sc = sc.fit_transform(X_train_logreg)

X_train_logreg_sc_no_nan = pd.DataFrame(X_train_logreg_sc)

X_train_logreg_sc_no_nan.dropna(axis=1, inplace=True)

In [18]:
logit = LogisticRegression(random_state=R_STATE, C=4.7)
logit.fit(X_train_logreg_sc_no_nan,y_trian_logreg)

LogisticRegression(C=4.7, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
X_scale_valid = sc.transform(X_valid)

X_test_logreg_sc_no_nan = pd.DataFrame(X_scale_valid)
X_test_logreg_sc_no_nan.dropna(axis=1, inplace=True)

predict_logit = logit.predict_proba(X_test_logreg_sc_no_nan)[:, 1]

roc_auc_score(y_valid, predict_logit)

0.6987599865150877

# Combo training

In [22]:
X_train_combo = pd.concat([pd.Series(predict_logit), pd.Series(predict_xgb)], axis=1)
X_train_combo.rename(columns={0: 'logit', 1: 'xgb'}, inplace=True)
X_train_combo.head()

,logit,xgb
0,0.129221,0.048074
1,0.086530,0.026434
2,0.280959,0.146254
3,0.452519,0.692166
4,0.184932,0.251610


In [0]:
def combo_prediction(X: pd.DataFrame, w=0.9) -> pd.Series:
  return X.xgb * w + X.logit * (1-w)

In [24]:
predict_combo = combo_prediction(X_train_combo, w=0.8)
roc_auc_score(y_valid, predict_combo)

0.7359754935838765

# Final training

In [26]:
xgb.fit(X_train_new.values, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.1,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=2, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=1.4, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsamle=0.9, subsample=1, verbosity=1)

In [27]:
X_train_sc = sc.fit_transform(X_train_new)

X_train_logreg_sc_no_nan = pd.DataFrame(X_train_sc)
X_train_logreg_sc_no_nan.dropna(axis=1, inplace=True)

logit.fit(X_train_logreg_sc_no_nan, y_train)

LogisticRegression(C=4.7, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=17, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Make Predict

In [28]:
mf_test = MakeFeatures(is_weekend=1, 
                  seasons=1, 
                  day_of_month=1, 
                  dep_capital=1, 
                  arr_capital=1, 
                  route=1, 
                  log_dist=1, 
                  uc=1, 
                  time = 1,
                  new_dep_time = 1,
                  hour_imp = 1)
mf_test.fit(test)

X_test = mf_test.fit_transform(test)
X_test.head()

Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Hour_sq-s feature complete!
Route complete!
Seasons complete!
IsWeekend complete!
TimeOfDay complete!
Day complete!
DepFromCap complete!
ArrivInCap complete!
LogDist complete!
UniqueCarrier dummies complete!
Hour Minute feature complete!
Hour_sq-s feature complete!
Route complete!


,Seasons_cos,Seasons_sin,IsWeekend,Day,DepFromCap,ArrForCap,LogDist,Hour,Minute,Hour_sq,Hour_sq2,9E,AA,AQ,AS,B6,CO,DL,EV,F9,FL,HA,MQ,NW,OH,OO,UA,US,WN,XE,YV,ABE_Origin,ABI_Origin,ABQ_Origin,ABY_Origin,ACK_Origin,ACT_Origin,ACV_Origin,ACY_Origin,ADK_Origin,...,SLE_Dest,SMF_Dest,SMX_Dest,SNA_Dest,SOP_Dest,SPI_Dest,SPS_Dest,SRQ_Dest,STL_Dest,STT_Dest,STX_Dest,SUN_Dest,SUX_Dest,SWF_Dest,SYR_Dest,TLH_Dest,TOL_Dest,TPA_Dest,TRI_Dest,TTN_Dest,TUL_Dest,TUP_Dest,TUS_Dest,TVC_Dest,TWF_Dest,TXK_Dest,TYR_Dest,TYS_Dest,VLD_Dest,VPS_Dest,WRG_Dest,WYS_Dest,XNA_Dest,YAK_Dest,YKM_Dest,YUM_Dest,day,evening,morning,night
0,-8.660254e-01,-5.000000e-01,0,25,0,0,6.393591,6,15,36,1296,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-5.000000e-01,8.660254e-01,0,17,0,0,7.118826,7,39,49,2401,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1.000000e+00,-2.449294e-16,1,2,0,0,6.357842,6,51,36,1296,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,6.123234e-17,1.000000e+00,1,25,1,0,5.932245,16,14,256,65536,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,-1.000000e+00,1.224647e-16,0,6,0,0,5.552960,15,5,225,50625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [0]:
drop_cols = list(set(X_test.columns) - set(X_train_xgb.columns))
X_test.drop(drop_cols, axis=1, inplace=True)

In [0]:
null_cols = list(set(X_train_xgb.columns) - set(X_test.columns))

for col in null_cols:
  X_test[col] = 0

In [0]:
neworder = X_train_new.columns
X_test_new = X_test.reindex(columns=neworder)

In [32]:
X_test_new.head()

,Seasons_cos,Seasons_sin,IsWeekend,Day,DepFromCap,ArrForCap,LogDist,Hour,Minute,Hour_sq,Hour_sq2,AA,AQ,AS,B6,CO,DH,DL,EV,F9,FL,HA,HP,MQ,NW,OH,OO,TZ,UA,US,WN,XE,YV,ABE_Origin,ABI_Origin,ABQ_Origin,ABY_Origin,ACK_Origin,ACT_Origin,ACV_Origin,...,SMF_Dest,SMX_Dest,SNA_Dest,SOP_Dest,SPI_Dest,SPS_Dest,SRQ_Dest,STL_Dest,STT_Dest,STX_Dest,SUN_Dest,SWF_Dest,SYR_Dest,TEX_Dest,TLH_Dest,TOL_Dest,TPA_Dest,TRI_Dest,TTN_Dest,TUL_Dest,TUP_Dest,TUS_Dest,TVC_Dest,TWF_Dest,TXK_Dest,TYR_Dest,TYS_Dest,VCT_Dest,VIS_Dest,VLD_Dest,VPS_Dest,WRG_Dest,WYS_Dest,XNA_Dest,YAK_Dest,YUM_Dest,day,evening,morning,night
0,-8.660254e-01,-5.000000e-01,0,25,0,0,6.393591,6,15,36,1296,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-5.000000e-01,8.660254e-01,0,17,0,0,7.118826,7,39,49,2401,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1.000000e+00,-2.449294e-16,1,2,0,0,6.357842,6,51,36,1296,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,6.123234e-17,1.000000e+00,1,25,1,0,5.932245,16,14,256,65536,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,-1.000000e+00,1.224647e-16,0,6,0,0,5.552960,15,5,225,50625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [0]:
predict_xgb_test = xgb.predict_proba(X_test_new.values)[:, 1]



In [0]:
X_test_new_logit = sc.transform(X_test_new)

X_test_new_logit_no_nan = pd.DataFrame(X_test_new_logit)
X_test_new_logit_no_nan.dropna(axis=1, inplace=True)

predict_logit_test = logit.predict_proba(X_test_new_logit_no_nan)[:, 1]

In [36]:
X_test_combo = pd.concat([pd.Series(predict_logit_test), pd.Series(predict_xgb_test)], axis=1)
X_test_combo.rename(columns={0: 'logit', 1: 'xgb'}, inplace=True)
X_test_combo.head()

,logit,xgb
0,0.053925,0.039990
1,0.148032,0.059010
2,0.049038,0.031854
3,0.262468,0.190930
4,0.253897,0.251239


In [0]:
predict_kaggle = combo_prediction(X_test_combo, w=0.8)

In [0]:
make_predict(predict_kaggle, 'combo_all_dataset_final')

In [39]:
predict_kaggle

0        0.042777
1        0.076814
2        0.035291
3        0.205238
4        0.251771
           ...   
99995    0.058079
99996    0.246447
99997    0.211538
99998    0.114251
99999    0.095142
Length: 100000, dtype: float64